# **Third Attempt at generating text using RNN**

So far i've tried to generate shakespeare like text using Tensorflow. ith my first few attempts i went of the Udacity course and tried to train my own model to perfom text generation. I didn't get very far as the models i tried to train were far to complex and i saw very little results. Following the guide on tensorflow, the trained model was far simplier and was successfully trained.

I thought i'd give it another go at training my own text generation model/Something a bit intresting...

Text Generation Model trained on [Anime Quotes](https://www.kaggle.com/datasets/tarundalal/anime-quotes)


P.s
I'm most likely going to steal some stuff from both the Udacity and Tensorflow guide

# **Import Dependencies**

In [2]:
import tensorflow as tf
import numpy as np
import urllib.request
import csv

print(tf.__version__)


2.8.2


# **Download the dataset**

In [3]:
# i downloaded the dataset from this link
url = "https://www.kaggle.com/datasets/tarundalal/anime-quotes/download?datasetVersionNumber=1"


In [ ]:
!pwd

Extracted the csv file and loaded it into the contents folder

In [4]:
# read the csvfile
anime_quotes = []

# the csv file contains Quote, character, Anime. For this task we are only
# interested in the quote so we would only get the first column from each row.
with open('AnimeQuotes.csv') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  for row in csv_reader:
    anime_quotes.append(row[0])

print(anime_quotes[:10])


['Quote', 'People’s lives don’t end when they die, it ends when they lose faith.', 'If you don’t take risks, you can’t create a future!', 'If you don’t like your destiny, don’t accept it.', 'When you give up, that’s when the game ends.', 'All we can do is live until the day we die. Control what we can…and fly free.', 'Forgetting is like a wound. The wound may heal, but it has already left a scar.', 'It’s just pathetic to give up on something before you even give it a shot.”', 'If you don’t share someone’s pain, you can never understand them.', 'Whatever you lose, you’ll find it again. But what you throw away you’ll never get back.']


In [5]:
# remove the header
anime_quotes = anime_quotes[1:]

print(anime_quotes[:10])
print(len(anime_quotes))


['People’s lives don’t end when they die, it ends when they lose faith.', 'If you don’t take risks, you can’t create a future!', 'If you don’t like your destiny, don’t accept it.', 'When you give up, that’s when the game ends.', 'All we can do is live until the day we die. Control what we can…and fly free.', 'Forgetting is like a wound. The wound may heal, but it has already left a scar.', 'It’s just pathetic to give up on something before you even give it a shot.”', 'If you don’t share someone’s pain, you can never understand them.', 'Whatever you lose, you’ll find it again. But what you throw away you’ll never get back.', 'We don’t have to know what tomorrow holds! That’s why we can live for everything we’re worth today!”']
121


# **Prepare the text**

The main task is here is to be able to generate anime quotes from our own seed text. Towards this we need, a set of feature and labels to train the model on.

<br>

**Set features and labels**   
The feature and labels need to reflect the task, so the feature should be a set of initial text and the label should be the next set of text.

From what i've seen there are 2 ways we can approach this, we can create a model which Predicts the next char or predicts the next word. I'll try out the different methods to prepare the text
- Predicting next char 
- Predicting the next probable word


In this collab i'll generate a model to predict the next probable char.



In [6]:
# combine the contents of the list into a single string
all_anime_quotes = " ".join(anime_quotes)


num_char = len(all_anime_quotes)
unique_chars = set(all_anime_quotes)
vocab_size = len(unique_chars)

print(all_anime_quotes)
print(f"Unique_chars: {unique_chars}")
print(f"Total number of charachthers in all_anime_quotes: {num_char}")
print(f"Vocabulary size: {vocab_size}")


People’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you can’t create a future! If you don’t like your destiny, don’t accept it. When you give up, that’s when the game ends. All we can do is live until the day we die. Control what we can…and fly free. Forgetting is like a wound. The wound may heal, but it has already left a scar. It’s just pathetic to give up on something before you even give it a shot.” If you don’t share someone’s pain, you can never understand them. Whatever you lose, you’ll find it again. But what you throw away you’ll never get back. We don’t have to know what tomorrow holds! That’s why we can live for everything we’re worth today!” Why should I apologize for being a monster? Has anyone ever apologized for turning me into one? People become stronger because they have memories they can’t forget. I’ll leave tomorrow’s problems to tomorrow’s me. If you wanna make people dream, you’ve gotta start by believing in that dream you

In [7]:
print(sorted(unique_chars))


[' ', '!', ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '’', '“', '”', '…']


To recap the steps we are going to take for the text generation model.

**Preparing the text**
- We are going to perform tokenization on each individual chars to convert them into tokens
- From the tokens we would then create sequences. We would create sequences of length 100 which would be our feature. Our label would be our sequence shifted one way to the right.

**Model training**
- We would then train an RNN model on the features and labels.

**Text generation**
- We would then generate a text from a seed word using the trained model

define a function to map the char into tokens.

In [8]:
# define a dictionary to map the char into token
char_to_token = dict([(char, token) for token, char in enumerate(unique_chars)])
print(char_to_token)


{'A': 0, 'W': 1, 'g': 2, 'u': 3, 'h': 4, 'P': 5, 'l': 6, 'R': 7, 'M': 8, '\xa0': 9, 'f': 10, '”': 11, '…': 12, 'N': 13, '-': 14, '?': 15, 'e': 16, 'I': 17, 'C': 18, 't': 19, 'E': 20, 'U': 21, 'i': 22, 'n': 23, ',': 24, 'm': 25, 'O': 26, 'o': 27, 'w': 28, 'F': 29, 'c': 30, 'S': 31, 'V': 32, '!': 33, 'j': 34, 'J': 35, '’': 36, 'B': 37, ':': 38, 'G': 39, '“': 40, 'x': 41, 'T': 42, 'a': 43, 'q': 44, 's': 45, 'z': 46, 'D': 47, 'K': 48, 'p': 49, ' ': 50, '.': 51, 'H': 52, 'r': 53, 'b': 54, 'd': 55, 'v': 56, 'y': 57, 'L': 58, 'k': 59, 'Y': 60}


In [9]:
# create a dictionary with inverted mapping
token_to_char = dict([(token, char) for char, token in char_to_token.items()])
print(token_to_char)


{0: 'A', 1: 'W', 2: 'g', 3: 'u', 4: 'h', 5: 'P', 6: 'l', 7: 'R', 8: 'M', 9: '\xa0', 10: 'f', 11: '”', 12: '…', 13: 'N', 14: '-', 15: '?', 16: 'e', 17: 'I', 18: 'C', 19: 't', 20: 'E', 21: 'U', 22: 'i', 23: 'n', 24: ',', 25: 'm', 26: 'O', 27: 'o', 28: 'w', 29: 'F', 30: 'c', 31: 'S', 32: 'V', 33: '!', 34: 'j', 35: 'J', 36: '’', 37: 'B', 38: ':', 39: 'G', 40: '“', 41: 'x', 42: 'T', 43: 'a', 44: 'q', 45: 's', 46: 'z', 47: 'D', 48: 'K', 49: 'p', 50: ' ', 51: '.', 52: 'H', 53: 'r', 54: 'b', 55: 'd', 56: 'v', 57: 'y', 58: 'L', 59: 'k', 60: 'Y'}


In [10]:
# Sanity check
print(f"A has token {char_to_token['A']}")
print(f"{char_to_token['A']} represents {token_to_char[char_to_token['A']]}")


A has token 0
0 represents A


In [11]:
# Convert the text data into sequences
sequences = []
for char in all_anime_quotes:
  token = char_to_token[char]
  sequences.append(token)

print(sequences)
print(f"Length of sequence: {len(sequences)}")


[5, 16, 27, 49, 6, 16, 36, 45, 50, 6, 22, 56, 16, 45, 50, 55, 27, 23, 36, 19, 50, 16, 23, 55, 50, 28, 4, 16, 23, 50, 19, 4, 16, 57, 50, 55, 22, 16, 24, 50, 22, 19, 50, 16, 23, 55, 45, 50, 28, 4, 16, 23, 50, 19, 4, 16, 57, 50, 6, 27, 45, 16, 50, 10, 43, 22, 19, 4, 51, 50, 17, 10, 50, 57, 27, 3, 50, 55, 27, 23, 36, 19, 50, 19, 43, 59, 16, 50, 53, 22, 45, 59, 45, 24, 50, 57, 27, 3, 50, 30, 43, 23, 36, 19, 50, 30, 53, 16, 43, 19, 16, 50, 43, 50, 10, 3, 19, 3, 53, 16, 33, 50, 17, 10, 50, 57, 27, 3, 50, 55, 27, 23, 36, 19, 50, 6, 22, 59, 16, 50, 57, 27, 3, 53, 50, 55, 16, 45, 19, 22, 23, 57, 24, 50, 55, 27, 23, 36, 19, 50, 43, 30, 30, 16, 49, 19, 50, 22, 19, 51, 50, 1, 4, 16, 23, 50, 57, 27, 3, 50, 2, 22, 56, 16, 50, 3, 49, 24, 50, 19, 4, 43, 19, 36, 45, 50, 28, 4, 16, 23, 50, 19, 4, 16, 50, 2, 43, 25, 16, 50, 16, 23, 55, 45, 51, 50, 0, 6, 6, 50, 28, 16, 50, 30, 43, 23, 50, 55, 27, 50, 22, 45, 50, 6, 22, 56, 16, 50, 3, 23, 19, 22, 6, 50, 19, 4, 16, 50, 55, 43, 57, 50, 28, 16, 50, 55, 22, 16,

In [12]:
# From the sequence_data create a list containing
sequence_length = 100
sequences_as_tf_data = tf.data.Dataset.from_tensor_slices(sequences).batch(sequence_length+1, drop_remainder=True)


In [13]:
# display the 2 samples from the dataset
for sample in sequences_as_tf_data.take(2):
  print("".join([token_to_char[token] for token in sample.numpy()]))

People’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you ca
n’t create a future! If you don’t like your destiny, don’t accept it. When you give up, that’s when t


In [14]:
# Split sequences into features and labels
def split_sequence(sequence):
  feature = sequence[:-1]
  label = sequence[1:]
  return feature, label


In [15]:
# try it out on the 2 samples
for sample in sequences_as_tf_data.take(2):
  (feature, label) = split_sequence(sample.numpy())
  print("\nFeature, Label pair")
  print("".join([token_to_char[token] for token in feature]))
  print("".join([token_to_char[token] for token in label]))



Feature, Label pair
People’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you c
eople’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you ca

Feature, Label pair
n’t create a future! If you don’t like your destiny, don’t accept it. When you give up, that’s when 
’t create a future! If you don’t like your destiny, don’t accept it. When you give up, that’s when t


In [16]:
# Apply the split_sequence function to the dataset
feature_label_data = sequences_as_tf_data.map(split_sequence)

for feature, label in feature_label_data.take(1):
  print(feature)
  print(label)


tf.Tensor(
[ 5 16 27 49  6 16 36 45 50  6 22 56 16 45 50 55 27 23 36 19 50 16 23 55
 50 28  4 16 23 50 19  4 16 57 50 55 22 16 24 50 22 19 50 16 23 55 45 50
 28  4 16 23 50 19  4 16 57 50  6 27 45 16 50 10 43 22 19  4 51 50 17 10
 50 57 27  3 50 55 27 23 36 19 50 19 43 59 16 50 53 22 45 59 45 24 50 57
 27  3 50 30], shape=(100,), dtype=int32)
tf.Tensor(
[16 27 49  6 16 36 45 50  6 22 56 16 45 50 55 27 23 36 19 50 16 23 55 50
 28  4 16 23 50 19  4 16 57 50 55 22 16 24 50 22 19 50 16 23 55 45 50 28
  4 16 23 50 19  4 16 57 50  6 27 45 16 50 10 43 22 19  4 51 50 17 10 50
 57 27  3 50 55 27 23 36 19 50 19 43 59 16 50 53 22 45 59 45 24 50 57 27
  3 50 30 43], shape=(100,), dtype=int32)


We have our text data prepared. Inputs to the model is a sequence of tokens and the label is also the same sequence shifted by 1 to the right.

In [17]:
# might be easier to convert sequences to strings if we define a function to convert it

def convert_sequence_to_string(sequence):
  string = "".join([token_to_char[token] for token in sequence])
  return string


In [18]:
for feature, label in feature_label_data.take(1):
  print(convert_sequence_to_string(feature.numpy()))
  print(convert_sequence_to_string(label.numpy()))

People’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you c
eople’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you ca


In [19]:
# create a batched dataset
batched_dataset = (feature_label_data.batch(1))

Althought this seems un-necessary considering that, there is really only 124 samples, RNN models expect a batch dimension. 

# **Define the RNN model**

In [57]:
vocab_szie = 61
Embedding_dim = 128
GRU_units = 256


In [58]:
# define a text generation model
Anime_qoutes_model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, Embedding_dim),
                                         tf.keras.layers.GRU(units=GRU_units, dropout=0.5, 
                                                             recurrent_dropout=0.25,
                                                             return_sequences=True),
                                         tf.keras.layers.Dense(units=vocab_size, activation="softmax")])


In [59]:
# Compile the model
Anime_qoutes_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           )

Anime_qoutes_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         7808      
                                                                 
 gru_1 (GRU)                 (None, None, 256)         296448    
                                                                 
 dense_1 (Dense)             (None, None, 61)          15677     
                                                                 
Total params: 319,933
Trainable params: 319,933
Non-trainable params: 0
_________________________________________________________________


Some notes missed out from an unsaved version
- Batch dimension in the data is needed for RNN 
- SparseCategoricalCrossentropy used for multiclass classification when expected labels are integers and not one-hot encoded labels.
- Use categoricalCrossentropy loss for multiclass classifications with one-hot encoded labels

# **Train the model**

**Run forward pass and get the loss**

In [60]:
# get a single feature and label from the batch dataset
for batch_feature, batch_label in batched_dataset.take(1):
  print(batch_feature[0])
  print(batch_label[0])
  

tf.Tensor(
[ 5 16 27 49  6 16 36 45 50  6 22 56 16 45 50 55 27 23 36 19 50 16 23 55
 50 28  4 16 23 50 19  4 16 57 50 55 22 16 24 50 22 19 50 16 23 55 45 50
 28  4 16 23 50 19  4 16 57 50  6 27 45 16 50 10 43 22 19  4 51 50 17 10
 50 57 27  3 50 55 27 23 36 19 50 19 43 59 16 50 53 22 45 59 45 24 50 57
 27  3 50 30], shape=(100,), dtype=int32)
tf.Tensor(
[16 27 49  6 16 36 45 50  6 22 56 16 45 50 55 27 23 36 19 50 16 23 55 50
 28  4 16 23 50 19  4 16 57 50 55 22 16 24 50 22 19 50 16 23 55 45 50 28
  4 16 23 50 19  4 16 57 50  6 27 45 16 50 10 43 22 19  4 51 50 17 10 50
 57 27  3 50 55 27 23 36 19 50 19 43 59 16 50 53 22 45 59 45 24 50 57 27
  3 50 30 43], shape=(100,), dtype=int32)


In [61]:
# display the string
print(convert_sequence_to_string(batch_feature[0].numpy()))
print(convert_sequence_to_string(batch_label[0].numpy()))


People’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you c
eople’s lives don’t end when they die, it ends when they lose faith. If you don’t take risks, you ca


In [62]:
# pass the feature to the untrained model and get the prediction
batch_prediction = Anime_qoutes_model(batch_feature)
print(batch_prediction)


tf.Tensor(
[[[0.01622223 0.01648524 0.01629859 ... 0.01644206 0.01629681 0.016256  ]
  [0.01631861 0.01654662 0.01647236 ... 0.01672636 0.01627488 0.01639478]
  [0.01657045 0.01644407 0.01662751 ... 0.01667407 0.01624372 0.01650744]
  ...
  [0.0166453  0.01644128 0.01629046 ... 0.01627394 0.01646843 0.01657229]
  [0.01669368 0.01642527 0.01640781 ... 0.01635736 0.01651504 0.01682277]
  [0.01652392 0.01665036 0.01662724 ... 0.01645394 0.0164212  0.01659049]]], shape=(1, 100, 61), dtype=float32)


In [63]:
# something intresting to show
# try passing just the feature and not the batch feature.
try:
  batch_prediction = Anime_qoutes_model(batch_feature[0])
  print(batch_prediction)
except Exception as e:
  print("Sorry that's a no no")
  print(f"{e}")

Sorry that's a no no
Exception encountered when calling layer "sequential_1" (type Sequential).

Input 0 of layer "gru_1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (100, 128)

Call arguments received:
  • inputs=tf.Tensor(shape=(100,), dtype=int32)
  • training=False
  • mask=None


The GRU layer expects a feature with 3 dimensions: `[batch dimension, time steps, time step dimension]`

In [64]:
# convert the prediction into a readable text
predicted_char_list = []
for prob_distribution in batch_prediction[0].numpy():
  char_id = np.argmax(prob_distribution)
  predicted_char_list.append(token_to_char[char_id])

print(predicted_char_list)


[',', '’', 'M', 'i', 'V', '’', 'v', 'M', 'm', 'm', '’', 'b', 'M', 'M', 'm', ' ', 'h', 'M', 'g', 'b', 'Y', 'M', 'M', 'M', ',', ',', '”', 'L', 'M', 'Y', '”', '”', 'g', '’', '’', '.', '’', '’', 't', ',', '’', 'b', 'Y', '’', 'M', 'M', ' ', ' ', ',', 'Y', 'L', 'M', 'Y', '”', '”', 'g', '’', '’', '’', 'A', 'M', 'M', 'Y', 'M', 'g', 'b', 'b', 't', 'f', 'r', 'f', 'V', 'Y', '’', 'A', '?', 'Y', '.', 'h', 'M', 'c', 'b', 'Y', '”', 'g', 'I', '’', '’', '”', 'b', 'B', 't', 'O', 't', 'Y', '’', 'T', '’', 'Y', 'h']


In [67]:
predicted_text = "".join(predicted_char_list)
print(predicted_text)
print(len(predicted_text))


,’MiV’vMmm’bMMm hMgbYMMM,,”LMY””g’’.’’t,’bY’MM  ,YLMY””g’’’AMMYMgbbtfrfVY’A?Y.hMcbY”gI’’”bBtOtY’T’Yh
100


In [69]:
# get the loss of the model
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
example_loss = loss(batch_label, batch_prediction).numpy()
print(example_loss)


4.110419


Sort of need to make sense of the warning. I've found this [thread](https://stackoverflow.com/questions/67848962/selecting-loss-and-metrics-for-tensorflow-model) on stackoverflow that provides some better guidance on selecting loss, metrics and difference between softmax and sigmoid activation.

**TLDR Summary**
- Can sparse_categorical_accuracy as a metric for classification task, where the label is an integer and not a one-hot encoded label.
- Similarly, we can use spare_categorical_crossentropy as a loss function for classification task in the same scenario as above.

<br>

- In cases, where we have our labels represented as one-hot encoded vectors we could use categorical_accuracy as a metric and categorical_crossentropy as a loss function.

<br>

- softmax activation functions are commonly used as the activation function in the output layer for the classification task. These functions produce a probabilitiy distribution, so the sum of the output from the layer = 1.
Generally if the model outputs a probability distribution, you'll need to set the from_logits = False.

<br>

So what are Logits???   
[Another stack overflow thread](https://stackoverflow.com/questions/34240703/what-are-logits-what-is-the-difference-between-softmax-and-softmax-cross-entrop)

**Summary**



In [71]:
# define the loss without setting the from_logits argument to be true
loss=tf.keras.losses.SparseCategoricalCrossentropy()
example_loss = loss(batch_label, batch_prediction).numpy()
print(example_loss)


4.110419


Happy days.

As annoying as it is i think it is best i keep the errors in the notebook so that i and anyone who might be reading this can easily see the issues, lessons and solutions i came across.

--
**redefine the model**

In [72]:
Anime_qoutes_model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,
                                                                    Embedding_dim),
                                          tf.keras.layers.GRU(units=GRU_units,
                                                              dropout=0.5,
                                                              recurrent_dropout=0.25,
                                                              return_sequences=True),
                                          tf.keras.layers.Dense(units=vocab_size, activation="softmax")])

Anime_qoutes_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                           metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [73]:
# define the model call backs
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./model_checkpoints/model_epoch_{epoch}_loss_{loss}",
                                                               monitor='loss',
                                                               save_best_only=True)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.1, patience=4)

In [74]:
history = Anime_qoutes_model.fit(batched_dataset, epochs=22,
                                 callbacks=[model_checkpoint_callback, early_stopping_callback])

Epoch 1/22
123/123 [==============================] - ETA: 0s - loss: 3.0713 - sparse_categorical_accuracy: 0.2240

123/123 [==============================] - 42s 328ms/step - loss: 3.0713 - sparse_categorical_accuracy: 0.2240
Epoch 2/22
123/123 [==============================] - ETA: 0s - loss: 2.4610 - sparse_categorical_accuracy: 0.3342

123/123 [==============================] - 40s 324ms/step - loss: 2.4610 - sparse_categorical_accuracy: 0.3342
Epoch 3/22
123/123 [==============================] - ETA: 0s - loss: 2.3127 - sparse_categorical_accuracy: 0.3537

123/123 [==============================] - 39s 317ms/step - loss: 2.3127 - sparse_categorical_accuracy: 0.3537
Epoch 4/22
123/123 [==============================] - ETA: 0s - loss: 2.2167 - sparse_categorical_accuracy: 0.3702

123/123 [==============================] - 40s 326ms/step - loss: 2.2167 - sparse_categorical_accuracy: 0.3702
Epoch 5/22
123/123 [==============================] - ETA: 0s - loss: 2.1399 - sparse_categorical_accuracy: 0.3911

...

123/123 [==============================] - 41s 333ms/step - loss: 1.3685 - sparse_categorical_accuracy: 0.5902
Epoch 19/22
123/123 [==============================] - ETA: 0s - loss: 1.3345 - sparse_categorical_accuracy: 0.5925

123/123 [==============================] - 40s 328ms/step - loss: 1.3345 - sparse_categorical_accuracy: 0.5925
Epoch 20/22
123/123 [==============================] - ETA: 0s - loss: 1.2909 - sparse_categorical_accuracy: 0.6076

123/123 [==============================] - 41s 334ms/step - loss: 1.2909 - sparse_categorical_accuracy: 0.6076
Epoch 21/22
123/123 [==============================] - ETA: 0s - loss: 1.2551 - sparse_categorical_accuracy: 0.6138

123/123 [==============================] - 40s 326ms/step - loss: 1.2551 - sparse_categorical_accuracy: 0.6138
Epoch 22/22
123/123 [==============================] - ETA: 0s - loss: 1.2245 - sparse_categorical_accuracy: 0.6252

123/123 [==============================] - 40s 322ms/step - loss: 1.2245 - sparse_categorical_accuracy: 0.6252


i have a feeling the final model is going to overfit onto the dataset, might be best to stop at the 10th epoch

# **Generate text using the trained model**